In [1]:
import sys
sys.path.insert(0, '..')

import polars as pl
from pathlib import Path

# Load LDA topic assignments - this already has speechContent + topic info
processed_dir = Path('../data/processed')
df = pl.read_parquet(processed_dir / 'topic_document_assignments_lda.parquet')

print(f"✓ LDA Topic Assignments loaded!")
print(f"  Shape: {df.shape}")
print(f"  Topics: {df['dominant_topic'].n_unique()} unique topics")
print(f"\nReady for sentiment analysis!")

✓ LDA Topic Assignments loaded!
  Shape: (588776, 17)
  Topics: 15 unique topics

Ready for sentiment analysis!


In [2]:
# trying the sentiment API

from germansentiment import SentimentModel

model = SentimentModel()

texts = [
    "Mit keinem guten Ergebniss","Das ist gar nicht mal so gut",
    "Total awesome!","nicht so schlecht wie erwartet",
    "Der Test verlief positiv.","Sie fährt ein grünes Auto."]
       
result = model.predict_sentiment(texts)
print(result)

classes, probabilities = model.predict_sentiment(["das ist super"], output_probabilities = True) 
print(classes, probabilities)


c:\Users\olive\miniconda3\envs\parl_speech\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['negative', 'negative', 'positive', 'positive', 'neutral', 'neutral']
['positive'] [[['positive', 0.9761366844177246], ['negative', 0.02354043908417225], ['neutral', 0.0003229434078093618]]]


In [3]:
# Cut paragraphs longer than 300 words using native Polars (10-100x faster than map_elements)
import polars as pl
from src.utils.text_processing import trim_to_max_words_native, add_word_count

print("Trimming paragraphs longer than 300 words...")

# Apply trimming using native Polars operations (much faster than map_elements)
df_trimmed = trim_to_max_words_native(df, 'speechContent', max_words=300)

# Calculate word counts for statistics
df_trimmed = add_word_count(df_trimmed, 'speechContent', 'word_count')

print(f"\nTrimming complete!")
print(f"Total rows: {df_trimmed.shape[0]}")
print(f"\nWord count statistics after trimming:")
print(f"  Min words: {df_trimmed['word_count'].min()}")
print(f"  Max words: {df_trimmed['word_count'].max()}")
print(f"  Mean words: {df_trimmed['word_count'].mean():.1f}")
print(f"  Median words: {df_trimmed['word_count'].median():.1f}")

print(f"\nFirst few rows after trimming:")
print(df_trimmed.select(['id', 'word_count', 'speechContent']).head(5))

Trimming paragraphs longer than 300 words...

Trimming complete!
Total rows: 588776

Word count statistics after trimming:
  Min words: 1
  Max words: 300
  Mean words: 29.8
  Median words: 20.0

First few rows after trimming:
shape: (5, 3)
┌─────────┬────────────┬─────────────────────────────────┐
│ id      ┆ word_count ┆ speechContent                   │
│ ---     ┆ ---        ┆ ---                             │
│ i64     ┆ u32        ┆ str                             │
╞═════════╪════════════╪═════════════════════════════════╡
│ 1000550 ┆ 38         ┆ Sehr geehrter Herr Präsident F… │
│ 1000550 ┆ 1          ┆ ({0})                           │
│ 1000550 ┆ 140        ┆ Aber lassen Sie mich, bevor wi… │
│ 1000550 ┆ 68         ┆ Da komme ich zum SPD-Parteivor… │
│ 1000550 ┆ 1          ┆ ({1})                           │
└─────────┴────────────┴─────────────────────────────────┘


In [ ]:
# Sentiment Analysis - OPTIMIZED VERSION with Status Updates
from germansentiment import SentimentModel
from tqdm import tqdm
import polars as pl
import time
import gc
import torch
import json
from pathlib import Path

print("="*80)
print("SENTIMENT ANALYSIS - OPTIMIZED")
print("="*80)

# Configuration - set to 1.0 for full dataset, 0.01 for 1% sample
SAMPLE_FRACTION = 0.01  # Using 1% sample due to time constraints

if SAMPLE_FRACTION < 1.0:
    # Use stratified sampling to maintain topic/party distribution
    df_analyze = df_trimmed.group_by(['dominant_topic', 'factionId']).sample(
        fraction=SAMPLE_FRACTION, 
        seed=42,
        shuffle=True
    )
    print(f"Using stratified {SAMPLE_FRACTION*100:.0f}% sample: {df_analyze.shape[0]:,} documents")
    print(f"  Maintains distribution across topics and parties")
    
    # Store the sampled IDs for later filtering of full dataset
    sampled_ids = df_analyze.select(['id', 'paragraph_number']).unique()
    print(f"  Unique IDs in sample: {len(sampled_ids):,}")
else:
    df_analyze = df_trimmed
    sampled_ids = None
    print(f"Processing full dataset: {df_analyze.shape[0]:,} documents")

# Load model with GPU detection
print("\n[1/4] Loading sentiment model...")
model = SentimentModel()

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✓ Model loaded on: {device.upper()}")
if device == "cuda":
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    # Increase batch size for GPU
    batch_size = 256  # GPU can handle larger batches
else:
    batch_size = 128  # CPU optimized batch size (doubled from 64)

# Extract texts
texts = df_analyze['speechContent'].to_list()
total_batches = (len(texts) + batch_size - 1) // batch_size

# Pre-allocate result lists for efficiency
sentiments = [None] * len(texts)
probabilities_list = [None] * len(texts)

# Checkpoint configuration
checkpoint_dir = Path('../data/interim')
checkpoint_dir.mkdir(parents=True, exist_ok=True)
checkpoint_file = checkpoint_dir / 'sentiment_checkpoint.json'
checkpoint_interval = 50  # Save every 50 batches

print(f"\n[2/4] Configuration:")
print(f"  Total documents: {len(texts):,}")
print(f"  Batch size: {batch_size}")
print(f"  Total batches: {total_batches:,}")
print(f"  Checkpoint interval: every {checkpoint_interval} batches")
print(f"  Estimated time: ~{(total_batches * (1.5 if device == 'cpu' else 0.5)) / 60:.1f} minutes")

# Check for existing checkpoint
start_batch = 0
if checkpoint_file.exists():
    print(f"\n⚠️  Found existing checkpoint!")
    response = input("Resume from checkpoint? (y/n): ")
    if response.lower() == 'y':
        with open(checkpoint_file, 'r') as f:
            checkpoint_data = json.load(f)
        start_batch = checkpoint_data['last_batch'] + 1
        sentiments[:checkpoint_data['processed_docs']] = checkpoint_data['sentiments']
        probabilities_list[:checkpoint_data['processed_docs']] = checkpoint_data['probabilities']
        print(f"✓ Resuming from batch {start_batch} ({checkpoint_data['processed_docs']:,} docs already processed)")

# Process with progress bar
print(f"\n[3/4] Running sentiment analysis...")
start_time = time.time()
batch_times = []

pbar = tqdm(range(start_batch, total_batches), desc="Processing", unit="batch", 
            initial=start_batch, total=total_batches)

for batch_idx in pbar:
    batch_start_time = time.time()
    i = batch_idx * batch_size
    batch_texts = [t if t else "" for t in texts[i:i+batch_size]]
    
    # Process batch
    classes, probs = model.predict_sentiment(batch_texts, output_probabilities=True)
    
    # Store results efficiently
    for j, (cls, prob) in enumerate(zip(classes, probs)):
        idx = i + j
        if idx < len(texts):
            sentiments[idx] = cls
            probabilities_list[idx] = prob
    
    # Track batch timing
    batch_time = time.time() - batch_start_time
    batch_times.append(batch_time)
    
    # Calculate statistics
    docs_done = min((batch_idx + 1) * batch_size, len(texts))
    elapsed = time.time() - start_time
    docs_per_sec = docs_done / elapsed if elapsed > 0 else 0
    remaining_docs = len(texts) - docs_done
    eta_seconds = remaining_docs / docs_per_sec if docs_per_sec > 0 else 0
    
    # Calculate average batch time (last 10 batches)
    avg_batch_time = sum(batch_times[-10:]) / len(batch_times[-10:])
    
    # Enhanced progress display
    pbar.set_postfix({
        'docs': f'{docs_done:,}/{len(texts):,}',
        'speed': f'{docs_per_sec:.0f} docs/s',
        'batch': f'{batch_time:.2f}s',
        'ETA': f'{eta_seconds/60:.1f}min'
    })
    
    # Checkpoint saving
    if (batch_idx + 1) % checkpoint_interval == 0:
        checkpoint_data = {
            'last_batch': batch_idx,
            'processed_docs': docs_done,
            'sentiments': sentiments[:docs_done],
            'probabilities': [p.tolist() if hasattr(p, 'tolist') else p for p in probabilities_list[:docs_done]],
            'timestamp': time.time()
        }
        with open(checkpoint_file, 'w') as f:
            json.dump(checkpoint_data, f)
    
    # Memory cleanup every 100 batches
    if (batch_idx + 1) % 100 == 0:
        gc.collect()
        if device == "cuda":
            torch.cuda.empty_cache()

elapsed_total = time.time() - start_time

# Remove checkpoint file on successful completion
if checkpoint_file.exists():
    checkpoint_file.unlink()

# Add results to dataframe - store probabilities as JSON for efficiency
print(f"\n[4/4] Creating results dataframe...")
df_sentiment = df_analyze.with_columns([
    pl.Series('sentiment_class', sentiments),
    pl.Series('sentiment_probabilities', [json.dumps(p.tolist() if hasattr(p, 'tolist') else p) for p in probabilities_list])
])

print(f"\n" + "="*80)
print("SENTIMENT ANALYSIS COMPLETE!")
print("="*80)
print(f"  Total documents: {len(df_sentiment):,}")
print(f"  Processing time: {elapsed_total/60:.1f} minutes")
print(f"  Speed: {len(texts)/elapsed_total:.1f} docs/sec")
print(f"  Avg batch time: {sum(batch_times)/len(batch_times):.2f}s")
print(f"  Device used: {device.upper()}")

# Show sentiment distribution
print(f"\n📊 Sentiment Distribution:")
sentiment_counts = df_sentiment['sentiment_class'].value_counts().sort('count', descending=True)
for row in sentiment_counts.iter_rows(named=True):
    percentage = (row['count'] / len(df_sentiment)) * 100
    print(f"  {row['sentiment_class']:10s}: {row['count']:6,} ({percentage:5.1f}%)")

# Cleanup
del texts
gc.collect()
if device == "cuda":
    torch.cuda.empty_cache()


SENTIMENT ANALYSIS - OPTIMIZED
Using 1% sample: 5,887 documents

[1/4] Loading sentiment model...
✓ Model loaded on: CPU

[2/4] Configuration:
  Total documents: 5,887
  Batch size: 128
  Total batches: 46
  Checkpoint interval: every 50 batches
  Estimated time: ~1.1 minutes

[3/4] Running sentiment analysis...


Processing:  52%|█████▏    | 24/46 [17:28<15:52, 43.29s/batch, docs=3,072/5,887, speed=3 docs/s, batch=44.74s, ETA=16.0min]

In [ ]:
# Save sentiment analysis results - ONLY matching IDs from sample
import polars as pl
from pathlib import Path

print("="*80)
print("SAVING RESULTS - 1% SAMPLE")
print("="*80)

# Define output path
output_dir = Path('../data/processed')
output_dir.mkdir(parents=True, exist_ok=True)

# If we used sampling, filter the original full topic data to only include sampled IDs
if SAMPLE_FRACTION < 1.0:
    print(f"\n📌 Filtering topic data to match sentiment sample IDs...")
    print(f"  Original topic data: {df.shape[0]:,} rows")
    print(f"  Sentiment sample: {df_sentiment.shape[0]:,} rows")
    
    # Get the IDs from sentiment analysis sample
    sample_ids = df_sentiment.select(['id', 'paragraph_number'])
    
    # Filter the original topic data to only include these IDs
    df_filtered = df.join(
        sample_ids,
        on=['id', 'paragraph_number'],
        how='inner'
    )
    
    print(f"  Filtered topic data: {df_filtered.shape[0]:,} rows")
    print(f"  ✓ IDs match: {df_filtered.shape[0] == df_sentiment.shape[0]}")
    
    # Now merge sentiment results with the filtered topic data
    df_final = df_filtered.join(
        df_sentiment.select(['id', 'paragraph_number', 'sentiment_class', 'sentiment_probabilities']),
        on=['id', 'paragraph_number'],
        how='inner'
    )
    
    print(f"\n✓ Final merged data: {df_final.shape[0]:,} rows")
else:
    # Full dataset - just merge directly
    df_final = df_sentiment

# Save the final dataset with topic + sentiment (1% sample)
output_file = output_dir / 'df_sample_sentiment.parquet'
df_final.write_parquet(output_file)

print(f"\n✓ Sentiment analysis results saved!")
print(f"  File: {output_file}")
print(f"  Rows: {len(df_final):,}")
print(f"  Columns: {len(df_final.columns)}")
print(f"  Sample fraction: {SAMPLE_FRACTION*100:.0f}%")

# Show column list
print(f"\n📋 Columns in final dataset:")
for i, col in enumerate(df_final.columns, 1):
    print(f"  {i:2d}. {col}")

# Show sample with topic + sentiment
print(f"\n📊 Sample Results (Topic + Sentiment):")
print(df_final.select([
    'id', 'firstName', 'lastName', 'factionId', 
    'dominant_topic', 'sentiment_class'
]).head(10))

# Cross-tabulation: Sentiment by Topic
print(f"\n📊 Sentiment Distribution by Topic:")
cross_tab = df_final.group_by(['dominant_topic', 'sentiment_class']).len().sort(['dominant_topic', 'sentiment_class'])
print(cross_tab.head(20))

# Distribution by Party
print(f"\n📊 Sentiment Distribution by Party:")
party_cross_tab = df_final.group_by(['factionId', 'sentiment_class']).len().sort(['factionId', 'sentiment_class'])
print(party_cross_tab)

print(f"\n✅ Data ready for final analysis!")
print(f"⚠️  Remember: This is a {SAMPLE_FRACTION*100:.0f}% stratified sample maintaining topic/party distributions")


✓ Sentiment analysis results saved!
  File: ..\data\processed\df_sample_sentiment.parquet
  Rows: 412
  Columns: 26

📊 Sentiment Analysis Summary:
  Total paragraphs analyzed: 412

  Sentiment Distribution:
    neutral   : 261 ( 63.3%)
    positive  : 107 ( 26.0%)
    negative  :  44 ( 10.7%)

✓ Data ready for further analysis!
